# Comprehensive Health & Network Analysis

**Objective:** Analyze the relationship between physiological health metrics (Garmin) and digital network activity (RouterSense).

**Datasets:**
1.  **Garmin Health**: `output/garmin_parsed/garmin_minute_health_joined.csv`
    -   *Metrics*: Heart Rate, Stress Level, Body Battery, Respiration Rate, Steps, Calories.
    -   *Granularity*: Minute-level.
2.  **RouterSense Network**: `data/phone_overall_activities.csv`
    -   *Metrics*: Upload/Download Bytes, Packets, App Names, Domains.
    -   *Granularity*: Event-based (needs aggregation).

**Goal:** Strategize and implement a robust merge of these two distinct datasets.

## 1. Setup & Configuration

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pytz
import json

# Configuration
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# Timezone
EST = pytz.timezone('US/Eastern')

print("✓ Libraries loaded")

✓ Libraries loaded


## 2. Load & Inspect Garmin Data
This dataset has already been processed to include **Respiration Rate** and consolidated to minute-level.

In [2]:
GARMIN_FILE = '../output/garmin_parsed/garmin_minute_health_joined.csv'

print(f"⏳ Loading Garmin data from {GARMIN_FILE}...")
garmin_df = pd.read_csv(GARMIN_FILE)

# Convert to Datetime (EST)
garmin_df['datetime'] = pd.to_datetime(garmin_df['datetime'], utc=True).dt.tz_convert(EST)

print(f"📊 Garmin Data Summary:")
print(f"   Rows: {len(garmin_df):,}")
print(f"   Time Range: {garmin_df['datetime'].min()} to {garmin_df['datetime'].max()}")
print(f"   Columns: {list(garmin_df.columns)}")

# Check for duplicates
duplicates = garmin_df['datetime'].duplicated().sum()
print(f"   Duplicate timestamps: {duplicates}")

garmin_df.head()

⏳ Loading Garmin data from ../output/garmin_parsed/garmin_minute_health_joined.csv...
📊 Garmin Data Summary:
   Rows: 42,743
   Time Range: 2025-11-05 00:00:00-05:00 to 2025-12-03 14:09:00-05:00
   Columns: ['datetime', 'heart_rate', 'body_battery', 'date', 'time', 'hour', 'minute', 'day_of_week', 'respiration_rate', 'steps_cumulative', 'calories_cumulative', 'distance_meters_cumulative', 'steps_per_minute', 'calories_per_minute', 'stress_level']
   Duplicate timestamps: 1739


,datetime,heart_rate,body_battery,date,time,hour,minute,day_of_week,respiration_rate,steps_cumulative,calories_cumulative,distance_meters_cumulative,steps_per_minute,calories_per_minute,stress_level
0,2025-11-05 00:00:00-05:00,NaN,NaN,2025-11-05,00:00:00,0,0,Wednesday,NaN,NaN,NaN,NaN,0.0,0.0,NaN
1,2025-11-05 00:01:00-05:00,NaN,NaN,2025-11-05,00:01:00,0,1,Wednesday,13.99,NaN,NaN,NaN,0.0,0.0,69.0
2,2025-11-05 00:02:00-05:00,NaN,NaN,2025-11-05,00:02:00,0,2,Wednesday,13.28,NaN,NaN,NaN,0.0,0.0,69.0
3,2025-11-05 00:03:00-05:00,NaN,NaN,2025-11-05,00:03:00,0,3,Wednesday,13.67,NaN,NaN,NaN,0.0,0.0,71.0
4,2025-11-05 00:04:00-05:00,NaN,NaN,2025-11-05,00:04:00,0,4,Wednesday,13.35,NaN,NaN,NaN,0.0,0.0,63.0


## 3. Load & Inspect RouterSense Data
This is the raw network activity log. Timestamps are in Unix epoch format.

In [ ]:
ROUTERSENSE_FILE = '../data/phone_overall_activities.csv'

print(f"⏳ Loading RouterSense data from {ROUTERSENSE_FILE}...")
rs_df = pd.read_csv(ROUTERSENSE_FILE)

# Convert Timestamp to Datetime (EST)
rs_df['datetime'] = pd.to_datetime(rs_df['timestamp'], unit='s', utc=True).dt.tz_convert(EST)

print(f"📊 RouterSense Data Summary:")
print(f"   Rows: {len(rs_df):,}")
print(f"   Time Range: {rs_df['datetime'].min()} to {rs_df['datetime'].max()}")

# Check granularity
rs_df['minute'] = rs_df['datetime'].dt.floor('min')
unique_minutes = rs_df['minute'].nunique()
print(f"   Unique Minutes: {unique_minutes:,}")
print(f"   Avg Records per Minute: {len(rs_df) / unique_minutes:.2f}")

rs_df.head()

## 4. Data Alignment Strategy

### The Challenge
- **Garmin**: 1 row per minute (Clean).
- **RouterSense**: Multiple rows per minute (Raw events).

### The Plan
1.  **Aggregate RouterSense**: Group by minute to create a single row per minute, summing bytes/packets and listing active apps.
2.  **Outer Join**: Merge with Garmin on the minute timestamp, keeping all data.

Let's visualize the overlap before merging.

In [ ]:
# Visualize Overlap
plt.figure(figsize=(15, 6))

# Plot Garmin Data Presence (using Heart Rate as proxy)
plt.plot(garmin_df['datetime'], [1]*len(garmin_df), '|', color='green', label='Garmin Data', alpha=0.5)

# Plot RouterSense Data Presence
plt.plot(rs_df['datetime'], [0.8]*len(rs_df), '|', color='blue', label='RouterSense Data', alpha=0.5)

plt.yticks([])
plt.title('Data Availability & Overlap')
plt.legend()
plt.tight_layout()
plt.show()

## 5. RouterSense Aggregation (Preparation for Merge)
We need to compress the RouterSense data into one row per minute.

In [ ]:
# Aggregation Logic
rs_minute = rs_df.groupby('minute').agg({
    'upload_byte_count': 'sum',
    'download_byte_count': 'sum',
    'upload_packet_count': 'sum',
    'download_packet_count': 'sum',
    'device_ip_address': 'first', # Assuming mostly static
    # 'entity_name': lambda x: list(set(x.dropna())) # List unique apps
}).reset_index()

rs_minute.rename(columns={'minute': 'datetime'}, inplace=True)

print(f"✓ Aggregated RouterSense to {len(rs_minute):,} unique minutes")
rs_minute.head()

## 6. Ready to Merge?
Run the cell below to perform the merge when you are satisfied with the data preparation.

In [ ]:
# Perform Merge
# merged_df = pd.merge(garmin_df, rs_minute, on='datetime', how='outer')
# merged_df.sort_values('datetime', inplace=True)
# merged_df.head()